# Non si capisce niente. Risistema il file

In [ ]:
import keras
from torch.utils.data import DataLoader

from keras_tuner import HyperParameters
from dataset.dataset_loader import dataset_loader
import keras_tuner

from utils.my_tuner import HistoryDeletingBayesianOptimization
from models.structure.tunable_hypermodel import TunableHyperModel
from dataset.k_fold_dataset_wrapper import KFoldDatasetWrapper
import torch

In [ ]:
# Initial steps
hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((224, 224), is_grayscale=False)
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=32, shuffle=True)

# HyperTuning and search
Ho fissato il numero di layer ma vorrei renderlo libero

In [ ]:
from models.structure.learning_parameters.sgd_learning_parameters import SgdLearningParametersTunable
from models.simple_cnn.conv_net_family import TunableConvNetFamily
from models.structure.tunable_hypermodel import TunableHyperModel

# For now the optimizer is also fixed to SGD with these parameters:
hyperparameters.Fixed("lr", 1e-4)
hyperparameters.Fixed("momentum", 0.9)

# For now dropout layers are frozen to be disabled.
hyperparameters.Fixed("dropout_0", False)

project_name = "multi-layers"
project_directory = "cnn_search"

tuner = HistoryDeletingBayesianOptimization(
    TunableHyperModel(TunableConvNetFamily(), SgdLearningParametersTunable(1e-4), (3, 224, 224)),
    hyperparameters=hyperparameters,
    objective='val_loss',
    tune_new_entries=True,
    executions_per_trial=1,
    overwrite=False,
    directory=project_directory,
    max_trials=15,
    project_name=project_name
)

# Proviamo con random tuner e riduciamo la batch size da 32 a 16
Il Random tuner con totale di 60 iterazioni come suggerito da: https://web.archive.org/web/20160701182750/http://blog.dato.com/how-to-evaluate-machine-learning-models-part-4-hyperparameter-tuning

In [ ]:
# Initial steps
hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((224, 224), is_grayscale=False)
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, batch_size=16, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=16, shuffle=True)

In [ ]:
# For now the optimizer is also fixed to SGD with these parameters:
hyperparameters.Fixed("lr", 1e-4)
hyperparameters.Fixed("momentum", 0.9)

hyperparameters.Int(f"filters_0", min_value=16, max_value=256, step=2, sampling='log')
hyperparameters.Int(f"filters_1", min_value=16, max_value=256, step=2, sampling='log')
hyperparameters.Int(f"filters_2", min_value=16, max_value=256, step=2, sampling='log')
hyperparameters.Int(f"filters_3", min_value=16, max_value=256, step=2, sampling='log')

hyperparameters.Choice(f"kernel_0", values=[3, 5], default=3)
hyperparameters.Choice(f"kernel_1", values=[3, 5], default=3)
hyperparameters.Choice(f"kernel_2", values=[3, 5], default=3)
hyperparameters.Choice(f"kernel_3", values=[3, 5], default=3)

hyperparameters.Int(name=f"units_0", min_value=32, max_value=256, step=2, sampling='log')
hyperparameters.Int(name=f"units_1", min_value=32, max_value=256, step=2, sampling='log')
hyperparameters.Int(name=f"units_2", min_value=32, max_value=256, step=2, sampling='log')

# For now dropout layers are frozen to be disabled.
hyperparameters.Fixed("dropout_0", False)
hyperparameters.Fixed("dropout_1", False)
hyperparameters.Fixed("dropout_2", False)



In [ ]:
from utils.my_tuner import HistoryDeletingRandomSearch
from models.simple_cnn.conv_net_family import TunableConvNetFamily
from models.structure.learning_parameters.sgd_learning_parameters import SgdLearningParametersTunable
project_name = "random-search-rich-structure"
directory = "cnn_search"

hyperparameters = keras_tuner.HyperParameters()
tuner = HistoryDeletingRandomSearch(
    TunableHyperModel(TunableConvNetFamily(), SgdLearningParametersTunable(learning_rate=1e-4), (3, 224, 224)),
    hyperparameters=hyperparameters,
    objective='val_loss',
    tune_new_entries=True,
    executions_per_trial=1,
    overwrite=False,
    directory=directory,
    max_trials=60,
    project_name=project_name
)

In [ ]:
import callbacks.threshold_stop_cb

search_has_been_done = True  # To avoid overriding my stuff.
if not search_has_been_done:
    tuner.search(train_dataloader, epochs=12, validation_data=validation_dataloader,
                 callbacks=[
                     keras.callbacks.CSVLogger(f"{directory}/{project_name}/search.log", separator=",", append=True),
                     callbacks.threshold_stop_cb.ThresholdStopCallback(0.5, 4),
                 ])

Trial 1  was skipper for unknown reasons

In [ ]:
tuner.results_summary(5)

In [ ]:
import pandas

csv = pandas.read_csv(f"./cnn_search/{project_name}/search.log")
csv['tuner_iteration'] = 0

current_iteration = -1
for index, row in enumerate(csv.itertuples()):
    if csv.at[index, 'epoch'] == 0:
        current_iteration += 1
    csv.at[index, 'tuner_iteration'] = current_iteration

# 2 Tuner iterations are missing in my CSV. 
# Might the reason be unknown all we know that 42 and 47 are mapped to 40 and 45
best_dataframe = csv.query("tuner_iteration in [40, 8, 23, 45, 14]")

In [ ]:
import plotly.express as px

loss_figure = px.line(csv, x="epoch", y=["loss"], color="tuner_iteration", template="plotly_white",
                      markers=True)
loss_figure.update_layout(title="Loss in tuner search", xaxis_title="Epoch", yaxis_title="Loss")

In [ ]:
figure = px.line(csv, x="epoch", y=["val_loss"], color="tuner_iteration", template="plotly_white", markers=True)
figure.update_layout(title="Validation Loss in tuner search", xaxis_title="Epoch", yaxis_title="Loss")

In [ ]:
import plotly.express as px

loss_figure = px.line(best_dataframe, x="epoch", y=["loss"], color="tuner_iteration", template="plotly_white",
                      markers=True)
loss_figure.update_layout(title="Loss in tuner search", xaxis_title="Epoch", yaxis_title="Loss")

In [ ]:
figure = px.line(best_dataframe, x="epoch", y=["val_loss"], color="tuner_iteration", template="plotly_white",
                 markers=True)
figure.update_layout(title="Validation Loss in tuner search", xaxis_title="Epoch", yaxis_title="Loss")

In [ ]:
best_hyperparameters_references = [
    dict(iteration=40, hyperparameters_index=0),
    dict(iteration=8, hyperparameters_index=1),
]

[print(f"iteration:{i['iteration']}, hp: {tuner.get_best_hyperparameters(5)[i['hyperparameters_index']].values}") for i
 in best_hyperparameters_references]

## Test con modello 8 
Funziona ora la procedura. Risistema il file

In [ ]:
from models.zero_one_validation_loss import zero_one_loss_binary

# Model 8
model_family = TunableConvNetFamily()
model_family.load_parameters(tuner.get_best_hyperparameters(2)[1])
model_family.parameters_fixed = True

project_name = "random-search-best-8-hp-env"
directory = "cnn_search"

learning_parameters = HyperParameters()
learning_parameters.Choice(name="batch_size", values=[8, 16, 32, 64], default=16)
learning_parameters.Float(name="lr", min_value=1e-5, max_value=1e-3, sampling='log', step=2),
learning_parameters.Float(name="momentum", min_value=0.5, max_value=1, sampling='reverse_log', step=2)

batch_tuner = HistoryDeletingBayesianOptimization(
    TunableHyperModel(model_family, SgdLearningParametersTunable(learning_rate=1e-4),
                      (3, 224, 224), tune_batch=True, verbose=True),
    hyperparameters=learning_parameters,
    objective='val_loss',
    tune_new_entries=False,
    executions_per_trial=1,
    overwrite=False,
    directory=directory,
    metrics=['accuracy'],
    max_trials=10,
    project_name=project_name
)

In [ ]:
import callbacks.threshold_stop_cb

batch_tuner.search(train_dataloader, epochs=1, validation_data=validation_dataloader, callbacks=[
    keras.callbacks.CSVLogger(f"{directory}/{project_name}/search.log", separator=",", append=True),
    callbacks.threshold_stop_cb.ThresholdStopCallback(0.6, 4),
])

## Test con modello 40 (Most promising)

In [ ]:
# Search for best hyperparameters for model 4 and 8: Batch size, and SGD params

# Initial steps
hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((224, 224), is_grayscale=False)
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

# No batch size is fixed
local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, shuffle=True)

In [ ]:
from models.zero_one_validation_loss import zero_one_loss_binary

# Model 8
model_family = TunableConvNetFamily()
model_family.load_parameters(tuner.get_best_hyperparameters(2)[0])

project_name = "multi-layers-best-model-8-hp-env"
directory = "cnn_search"

learning_parameters = keras_tuner.HyperParameters()
learning_parameters.Choice(name="batch_size", values=[8, 16, 32, 64], default=16)
learning_parameters.Float(name="lr", min_value=1e-5, max_value=1e-3, sampling='log', step=2),
learning_parameters.Float(name="momentum", min_value=0.5, max_value=1, sampling='reverse_log', step=2)

tuner = HistoryDeletingBayesianOptimization(
    TunableModelFamilyHypermodel((3, 224, 224), TunableConvNetFamily(), tune_batch=True),
    hyperparameters=hyperparameters,
    objective='val_loss',
    tune_new_entries=False,
    executions_per_trial=1,
    overwrite=False,
    directory=directory,
    metrics=['accuracy', zero_one_loss_binary],
    max_trials=10,
    project_name=project_name
)

# Sezione separata: Ricerca piu narrow

In [ ]:
import pandas

csv = pandas.read_csv("./cnn_search/multi-layers/search.log")
csv['tuner_iteration'] = 0

current_iteration = 0
for index, row in enumerate(csv.itertuples()):
    if csv.at[index, 'epoch'] == 0:
        current_iteration += 1
    csv.at[index, 'tuner_iteration'] = current_iteration

best_dataframe = csv.query("tuner_iteration in [8, 4, 12, 2, 10]")

In [ ]:
import plotly.express as px

loss_figure = px.line(best_dataframe, x="epoch", y=["loss"], color="tuner_iteration", template="plotly_white",
                      markers=True)
loss_figure.update_layout(title="Loss in tuner search", xaxis_title="Epoch", yaxis_title="Loss")

In [ ]:
best_hyperparameters_references = [
    dict(iteration=8, hyperparameters_index=0),
    dict(iteration=4, hyperparameters_index=1),
]

[print(f"iteration:{i['iteration']}, hp: {tuner.get_best_hyperparameters(5)[i['hyperparameters_index']].values}") for i
 in best_hyperparameters_references]

In [ ]:
import plotly.express as px

loss_figure = px.line(best_dataframe, x="epoch", y=["loss"], color="tuner_iteration", template="plotly_white",
                      markers=True)
loss_figure.update_layout(title="Loss in tuner search", xaxis_title="Epoch", yaxis_title="Loss")

In [ ]:
figure = px.line(best_dataframe, x="epoch", y=["val_loss"], color="tuner_iteration", template="plotly_white",
                 markers=True)
figure.update_layout(title="Validation Loss in tuner search", xaxis_title="Epoch", yaxis_title="Loss")

In [ ]:
# 4 Looks promising and also 8 is good. We keep those two

In [ ]:
best_hyperparameters_references = [
    dict(iteration=8, hyperparameters_index=0),
    dict(iteration=4, hyperparameters_index=1),
]

[print(f"iteration:{i['iteration']}, hp: {tuner.get_best_hyperparameters(5)[i['hyperparameters_index']].values}") for i
 in best_hyperparameters_references]

In [ ]:
# Search for best hyperparameters for model 4 and 8: Batch size, and SGD params

# Initial steps
hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((224, 224), is_grayscale=False)
dataset_split_controller = KFoldDatasetWrapper(5)
dataset_split_controller.load_data(train)

# No batch size is fixed
local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, shuffle=True)

In [ ]:
# Model 8 Summary
model_family = TunableConvNetFamily()
model_family.load_parameters(tuner.get_best_hyperparameters(2)[0])

temp_model = model_family.make_model((3, 224, 224))
model_family.compile_model(temp_model)

temp_model.summary()

In [ ]:
from models.zero_one_validation_loss import zero_one_loss_binary

# Model 8
model_family = TunableConvNetFamily()
model_family.load_parameters(tuner.get_best_hyperparameters(2)[0])

project_name = "multi-layers-best-model-8-hp-env"
directory = "cnn_search"

learning_parameters = keras_tuner.HyperParameters()
learning_parameters.Choice(name="batch_size", values=[8, 16, 32, 64], default=16)
learning_parameters.Float(name="lr", min_value=1e-5, max_value=1e-3, sampling='log', step=2),
learning_parameters.Float(name="momentum", min_value=0.5, max_value=1, sampling='reverse_log', step=2)

tuner = HistoryDeletingBayesianOptimization(
    TunableModelFamilyHypermodel((3, 224, 224), TunableConvNetFamily(), tune_batch=True),
    hyperparameters=hyperparameters,
    objective='val_loss',
    tune_new_entries=False,
    executions_per_trial=1,
    overwrite=False,
    directory=directory,
    metrics=['accuracy', zero_one_loss_binary],
    max_trials=10,
    project_name=project_name
)

In [ ]:
import callbacks.threshold_stop_cb

tuner.search(train_dataloader, epochs=12, validation_data=validation_dataloader, callbacks=[
    keras.callbacks.CSVLogger(f"{directory}/{project_name}/search.log", separator=",", append=True),
    callbacks.threshold_stop_cb.ThresholdStopCallback(0.5, 4),
])

In [ ]:
from models.zero_one_validation_loss import zero_one_loss_binary

# Model 4
model_family = TunableConvNetFamily()
model_family.load_parameters(tuner.get_best_hyperparameters(2)[1])

learning_parameters = keras_tuner.HyperParameters()
learning_parameters.Choice(name="batch_size", values=[8, 16, 32, 64], default=16)
learning_parameters.Float(name="lr", min_value=1e-5, max_value=1e-3, sampling='log', step=2),
learning_parameters.Float(name="momentum", min_value=0.5, max_value=1, sampling='reverse_log', step=2)

project_name = "multi-layers-best-model-4-hp-env"
directory = "cnn_search"

tuner = HistoryDeletingBayesianOptimization(
    TunableModelFamilyHypermodel((3, 224, 224), TunableConvNetFamily(), tune_batch=True),
    hyperparameters=hyperparameters,
    objective='val_loss',
    tune_new_entries=False,
    executions_per_trial=1,
    overwrite=False,
    directory=directory,
    max_trials=10,
    metrics=['accuracy', zero_one_loss_binary],
    project_name=project_name
)

In [ ]:
import callbacks.threshold_stop_cb

tuner.search(train_dataloader, epochs=12, validation_data=validation_dataloader, callbacks=[
    keras.callbacks.CSVLogger(f"{directory}/{project_name}/search.log", separator=",", append=True),
    callbacks.threshold_stop_cb.ThresholdStopCallback(0.5, 4),
])